In [55]:
import json
eval_data = json.load(open("C:\\Users\\PC\\CODE\\WDM-AI-TEMIS\\data-finetune\\final_data\\pymupdf_json.json", "r", encoding="utf-8"))

In [56]:
idx = 6

ground_truth = eval_data[idx]["markdown_content"]
predict = eval_data[idx]["pymupdf_md"]

In [57]:
print(ground_truth)

|Climate data for Berea District[hide]|Climate data for Berea District[hide]|Climate data for Berea District[hide]|Climate data for Berea District[hide]|Climate data for Berea District[hide]|Climate data for Berea District[hide]|Climate data for Berea District[hide]|Climate data for Berea District[hide]|Climate data for Berea District[hide]|Climate data for Berea District[hide]|Climate data for Berea District[hide]|Climate data for Berea District[hide]|Climate data for Berea District[hide]|Climate data for Berea District[hide]|
|--|--|--|--|--|--|--|--|--|--|--|--|--|--|
|Month| Jan| Feb| Mar| Apr| May| Jun| Jul| Aug| Sep| Oct| Nov| Dec| Year|
|Mean dailymaximum °C (°F)|26(79)|27(81)|25(77)|23(73)|20(68)|17(63)|17(63)|20(68)|23(73)|27(81)|27(81)|28(82)|23(74)|
|Mean dailyminimum °C (°F)|15(59)|15(59)|13(55)|6(43)|6(43)|3(37)|1(34)|0(32)|5(41)|12(54)|14(57)|16(61)|9(48)|
|Average rainfallmm (inches)|76(3.0)|98(3.9)|55(2.2)|28(1.1)|50(2.0)|34(1.3)|0(0)|0(0)|0(0)|22(0.9)|103(4.1)|68(2.7)|

In [58]:
print(predict)

|Climate data for Berea District [hide]|Col2|Col3|Col4|Col5|Col6|Col7|Col8|Col9|Col10|Col11|Col12|Col13|Col14|
|---|---|---|---|---|---|---|---|---|---|---|---|---|---|
|Month|Jan|Feb M|ar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec|Year|
|Mean daily<br>maximum °C (°F)|26<br>(79)|27<br>(81) (|25<br>77)|23<br>(73)|20<br>(68)|17<br>(63)|17<br>(63)|20<br>(68)|23<br>(73)|27<br>(81)|27<br>(81)|28<br>(82)|23<br>(74)|
|Mean daily<br>minimum °C (°F)|15<br>(59)|15<br>(59) (|13<br>55)|6<br>(43)|6<br>(43)|3<br>(37)|1<br>(34)|0<br>(32)|5<br>(41)|12<br>(54)|14<br>(57)|16<br>(61)|9<br>(48)|
|Average rainfall<br>mm (inches)|76<br>(3.0)|98<br>(3.9) (|55<br>2.2)|28<br>(1.1)|50<br>(2.0)|34<br>(1.3)|0<br>(0)|0<br>(0)|0<br>(0)|22<br>(0.9)|103<br>(4.1)|68<br>(2.7)|534<br>(21.0|
|[5]<br>Source 1:|[5]<br>Source 1:|[5]<br>Source 1:|[5]<br>Source 1:|[5]<br>Source 1:|[5]<br>Source 1:|[5]<br>Source 1:|[5]<br>Source 1:|[5]<br>Source 1:|[5]<br>Source 1:|[5]<br>Source 1:|[5]<br>Source 1:|[5]<br>Source 1:|[5]<br>Source 1:|


In [62]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import os

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

if not OPENAI_API_KEY:
    print("Lỗi: OPENAI_API_KEY không được tìm thấy.")
else:
    llm = ChatOpenAI(
        api_key=OPENAI_API_KEY,
        model="gpt-4o-mini",
        temperature=0,
        model_kwargs={
            "response_format": {"type": "json_object"}
        }
    )
    print("DEBUG: LLM initialized with JSON mode.")

DEBUG: LLM initialized with JSON mode.


In [60]:
from langchain_core.prompts import PromptTemplate
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel
from langchain_core.output_parsers import JsonOutputParser

def evaluate_table_extraction(ground_truth: str, predict: str, llm_model = None, debug: bool = False) -> any: # Thay đổi kiểu trả về nếu cần
   parser = JsonOutputParser()

   prompt_template_str = """
   Evaluate the accuracy of a table extraction tool by comparing two markdown tables.

   Consider the following criteria:
   - **Content**: Ensure matches and events in GROUND TRUTH are accurately represented in PREDICT.
   - **Formatting**: Check formatting consistency, including spacing and alignment.
   - **Data Integrity**: Verify accurate data representation of dates, teams, scores, match titles, and attendance.
   - **Order**: Confirm event sequence consistency.

   # Steps

   1. **Content Comparison**: Match rows and confirm data accuracy (team names, scores, etc.).
   2. **Formatting Review**: Check column alignment and use of whitespace.
   3. **Data Integrity Check**: Identify missing data or inconsistencies.
   4. **Order Verification**: Ensure consistent event sequences.
   5. **Assessment**: Note discrepancies and provide a score (0-1).

   # Output Format

   You must respond with a JSON object containing only a score field with a float value between 0 and 1.
   For example:
   {{"score": 0.85}}

   ### GROUND TRUTH
   {ground_truth}

   ### PREDICT
   {predict}

   Remember: Your entire response must be a valid JSON object with only a "score" field containing a number between 0 and 1.
   """
   prompt = PromptTemplate.from_template(template=prompt_template_str)

   try:
      chain = prompt | llm_model | parser
      if debug:
         print("DEBUG: Chain created successfully.")

      invocation_payload = {
         "ground_truth": ground_truth,
         "predict": predict
      }
      if debug:
         print(f"DEBUG: Invoking chain with payload (first 100 chars of each): \nGround truth: {ground_truth[:100]}...\nPredict: {predict[:100]}...")

      # Để kiểm tra riêng lẻ LLM (bỏ qua parser tạm thời):
      # formatted_prompt = prompt.invoke(invocation_payload)
      # print(f"DEBUG: Formatted prompt sent to LLM:\n{formatted_prompt.to_string()}") # Hoặc .text tùy phiên bản
      # llm_output = llm_model.invoke(formatted_prompt)
      # print(f"DEBUG: Raw output from LLM:\n{llm_output}")
      # res = parser.parse(llm_output) # Parse thủ công nếu muốn kiểm tra parser

      res = chain.invoke(invocation_payload)
      if debug:
         print(f"DEBUG: Chain invocation successful. Result (res): {res}")
         print(f"DEBUG: Type of res: {type(res)}")
      return res

   except Exception as e:
      if debug:   
         print(f"ERROR in evaluate_table_extraction: {e}")
         import traceback
         traceback.print_exc() # In ra chi tiết lỗi và dòng gây lỗi
      return "" # Hoặc None, hoặc một giá trị báo lỗi cụ thể

In [48]:
a = evaluate_table_extraction(ground_truth, predict, llm, debug=True)
print(a)

DEBUG: Chain created successfully.
DEBUG: Invoking chain with payload (first 100 chars of each): 
Ground truth: |Climate data for Berea District[hide]|Climate data for Berea District[hide]|Climate data for Berea ...
Predict: |Climate data for Berea District [hide]|Col2|Col3|Col4|Col5|Col6|Col7|Col8|Col9|Col10|Col11|Col12|Co...
DEBUG: Chain invocation successful. Result (res): {'score': 0.75}
DEBUG: Type of res: <class 'dict'>
{'score': 0.75}


In [63]:
from tqdm import tqdm
total_score = 0


for sample in tqdm(eval_data):
    ground_truth = sample['markdown_content']
    predict = sample['pymupdf_md']
    score = evaluate_table_extraction(ground_truth, predict, llm)
    total_score += score['score']
    # print(score)
print(total_score / len(eval_data))


100%|██████████| 163/163 [02:59<00:00,  1.10s/it]

0.6951533742331291
